In [1]:
# -*- coding: utf-8 -*-
"""
Created on Thu Jul 25 15:03:11 2024

@author: Alp Altunsoy
"""

import pandas as pd 
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import time
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.circuit.library import ZZFeatureMap
from qiskit.circuit.library import RealAmplitudes
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from IPython.display import clear_output
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import json

# Veri yükleme
print("Importing CSV train and test files")
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

# Satır sayısını minimize etme
print("Minimizing rows of train and test datasets")
train_data = train_data.head(2000)
test_data = test_data.head(100)

# Bağımsız değişkenlerin kodlanması
print("Encoding independent variables")
le = LabelEncoder()
gender = le.fit_transform(train_data.iloc[:, 2])
gender = pd.DataFrame(data=gender, columns=["Gender"])
x_train = pd.concat([gender, train_data.iloc[:, 7:8], train_data.iloc[:, 22:24], train_data.iloc[:, 14:15]], axis=1)
x_test = pd.concat([gender, test_data.iloc[:, 7:8], test_data.iloc[:, 22:24], test_data.iloc[:, 14:15]], axis=1)

# Bağımlı değişkenlerin oluşturulması
print("Making dependent variables")
y_train = train_data.iloc[:, -1].values
y_test = test_data.iloc[:, -1].values

# Eksik değerlerin doldurulması
print("Imputing missing values")
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
x_train = imputer.fit_transform(x_train)
x_test = imputer.fit_transform(x_test)

# Verilerin ölçeklendirilmesi
print("X scaling is started")
sc = MinMaxScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

# Y verilerinin kodlanması
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

# Kuantum algoritması
print("Quantum algorithm is started")
num_features = x_train.shape[1]

# Özellik haritası ve ansatz çizimi
feature_map = ZZFeatureMap(feature_dimension=num_features, reps=1)
feature_map.decompose().draw(output="mpl", style="clifford", fold=20)

ansatz = RealAmplitudes(num_qubits=num_features, reps=3)
ansatz.decompose().draw(output="mpl", style="clifford", fold=20)

optimizer = COBYLA(maxiter=100)
sampler = Sampler()

objective_func_vals = []
plt.rcParams["figure.figsize"] = (12, 6)

def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

vqc = VQC(sampler=sampler, feature_map=feature_map, ansatz=ansatz, optimizer=optimizer, callback=callback_graph)

# Öğrenme işlemi
print("Learning is started")
start = time.time()
vqc.fit(x_train, y_train)
elapsed = time.time() - start

print(f"Training time: {round(elapsed)} seconds")

# Skorların hesaplanması
train_score_q4 = vqc.score(x_train, y_train)
test_score_q4 = vqc.score(x_test, y_test)

print(f"Quantum VQC on the training dataset: {train_score_q4:.2f}")
print(f"Quantum VQC on the test dataset:     {test_score_q4:.2f}")

# Sonuçların kaydedilmesi
scores = {
   "train_score": train_score_q4,
   "test_score": test_score_q4,
   "training time" : elapsed
}
with open("report.json", "w") as f:
    json.dump(scores, f)

print("Scores have been saved to 'vqc_scores.json'")


Importing CSV train and test files
Encoding independent variables
Encoding independent variables
Minimizing rows of  train dataset
Minimizing rows of  test dataset
Making dependent Variables
Making dependent Variables
X scaling is started
X scaling is started
Imputing missing values
Imputing missing values
Quantum algorithm is started
Finding features
